<a href="https://colab.research.google.com/github/teymour-aldridge/NN/blob/master/computer_vision/boat_classifier/pytorch/convnet_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial setup

## Colab-related stuff

In [0]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


## tensorboardX (tensorboard for PyTorch)

In [0]:
!pip install tensorboardX

## PyTorch installation  + set base directory + import matplotlib, os and shutil

In [0]:
# Install PyTorch
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

  HTTP error 403 while getting http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
  Could not install requirement torch==0.4.1 from http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl because of error 403 Client Error: Forbidden for url: http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
Could not install requirement torch==0.4.1 from http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl because of HTTP error 403 Client Error: Forbidden for url: http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl for URL http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl


In [0]:
import os
import shutil
import matplotlib.pyplot as plt
base_path = "/content/gdrive/My Drive/Computing/ML/Computer Vision/Boat Classification/data"

# Artificially enhance dataset

In [0]:
classes = ['buoy', 'cruise ship', 'ferry boat', 'freight boat', 'gondola', 'inflatable boat', 'kayak', 'paper boat', 'sailboat']

for c in classes:
  pass

## Train-test split

In [0]:
"""
# Train/Test split
# Class names (exactly what you'd expect)
classes = ['buoy', 'cruise ship', 'ferry boat', 'freight boat', 'gondola', 'inflatable boat', 'kayak', 'paper boat', 'sailboat']
# Get the number of files in each directory
file_lengths = [len(os.listdir("/content/gdrive/My Drive/Computing/ML/Computer Vision/Boat Classification/data/" + c)) for c in classes]
# For each class
for c in classes:
  files = os.listdir(os.path.join(base_path, c)) # Get all files
  for i in range(len(files) // 5): # Get 20% of the files
    src = os.path.join(base_path, c, files[i])
    dest = os.path.abspath(os.path.join(base_path, 'test', c))
    if not os.path.exists(dest):
      os.mkdir(dest)
    shutil.move(src, dest)
"""

'\n# Train/Test split\n# Class names (exactly what you\'d expect)\nclasses = [\'buoy\', \'cruise ship\', \'ferry boat\', \'freight boat\', \'gondola\', \'inflatable boat\', \'kayak\', \'paper boat\', \'sailboat\']\n# Get the number of files in each directory\nfile_lengths = [len(os.listdir("/content/gdrive/My Drive/Computing/ML/Computer Vision/Boat Classification/data/" + c)) for c in classes]\n# For each class\nfor c in classes:\n  files = os.listdir(os.path.join(base_path, c)) # Get all files\n  for i in range(len(files) // 5): # Get 20% of the files\n    src = os.path.join(base_path, c, files[i])\n    dest = os.path.abspath(os.path.join(base_path, \'test\', c))\n    if not os.path.exists(dest):\n      os.mkdir(dest)\n    shutil.move(src, dest)\n'

In [0]:
train_path = os.path.join(base_path, 'train')
test_path = os.path.join(base_path, 'test')
classes = os.listdir(train_path)
print(classes)

['inflatable boat', 'gondola', 'sailboat', 'freight boat', 'ferry boat', 'paper boat', 'kayak', 'cruise ship', 'buoy']


## Generate an index for the dataset

In [0]:
data_class = 'sailboat'
os.path.join(train_path, data_class) + '/*.jpg'

'/content/gdrive/My Drive/Computing/ML/Computer Vision/Boat Classification/data/train/sailboat/*.jpg'

In [0]:
import pandas as pd
import glob
train_index = pd.DataFrame(columns=['class', 'path'])
test_index = pd.DataFrame(columns=['class', 'path'])

for i, data_class in enumerate(os.listdir(train_path)): # For each class (there are 9)
  
  for file in glob.glob(os.path.join(train_path, data_class) + '/*.jpg'): # Get all JPEG files

    train_index = train_index.append({
        'class': i,
        'path': os.path.join(train_path, data_class, file)
    }, ignore_index = True)
    
    
for i, data_class in enumerate(os.listdir(test_path)): # For each class (there are 9)
  
  for file in glob.glob(os.path.join(test_path, data_class) + '/*.jpg'): # Get all JPEG files

    test_index = test_index.append({
        'class': i,
        'path': os.path.join(test_path, data_class, file)
    }, ignore_index=True)

## PIL error fixing
If *VERSION: 4.0.0* is printed then restart the runtime, and run it again (normally fixes the problem) as scikit-image requires a version of PIL (Python Imaging Library) greater than 4.1.1 (I think).

In [0]:
!pip uninstall -y Pillow
# install the new one
!pip install Pillow==5.3.0
# import the new one
import PIL
print("VERSION:", PIL.PILLOW_VERSION)

Uninstalling Pillow-5.3.0:
  Successfully uninstalled Pillow-5.3.0
  Using cached https://files.pythonhosted.org/packages/62/94/5430ebaa83f91cc7a9f687ff5238e26164a779cca2ef9903232268b0a318/Pillow-5.3.0-cp36-cp36m-manylinux1_x86_64.whl
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
fastai 1.0.46 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.


VERSION: 5.3.0


# Define dataset

In [0]:
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.utils import shuffle
from PIL import Image

"""
  _____        _                 _   
 |  __ \      | |               | |  
 | |  | | __ _| |_ __ _ ___  ___| |_ 
 | |  | |/ _` | __/ _` / __|/ _ \ __|
 | |__| | (_| | || (_| \__ \  __/ |_ 
 |_____/ \__,_|\__\__,_|___/\___|\__|
 (Dataset class, allows for an index to be passed.)                                   
"""


class IndexedImageDataset(Dataset):
    def __init__(self, index, device, transform=None):
        """
        Initializes an IndexedImageDataset, which reads an index from a pandas dataframe, and acts as a database wrapper.
        :param index: A pandas dataframe storing with a list of classes and their respective files.
        :param transform: A function, to be applied to the image (any function will do).
        """
        assert isinstance(index, pd.DataFrame)  # Check if the 
        self.index = shuffle(index)
        self.transform = transform
        self.device = device
        self.transform = transform

    def __len__(self):
        return len(self.index.index)  # Return the length of the dataset

    def __getitem__(self, n):
        x, y = self.index['path'].iloc[n], self.index['class'].iloc[n]  # Get x and y values for the neural network
        x = Image.open(x)
        x.show()
        if self.transform:
          x = self.transform(x)
        x.double()
        return x, y

In [0]:
train_image_dataset=IndexedImageDataset(train_index, device,
                                       transform=transforms.Compose([
                                           transforms.Resize((512, 512)),
                                           transforms.ToTensor(),
                                           transforms.Normalize((0.611, 0.611, 0.611), (0.14, 0.14, 0.14)),
                                       ]))
train_image_loader = DataLoader(
    train_image_dataset,
    batch_size=4,
    num_workers=1,
    shuffle=False
)

## Normalize dataset
Note that this assumes the data is normally distributed (I haven't checked).

In [0]:
# Commented out because it does not make sense to perform the same calculations many times
"""mean = 0.
std = 0.
nb_samples = 0.
for data, _ in train_image_loader:
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples"""

'mean = 0.\nstd = 0.\nnb_samples = 0.\nfor data, _ in train_image_loader:\n    batch_samples = data.size(0)\n    data = data.view(batch_samples, data.size(1), -1)\n    mean += data.mean(2).sum(0)\n    std += data.std(2).sum(0)\n    nb_samples += batch_samples\n\nmean /= nb_samples\nstd /= nb_samples'

In [0]:
# print([mean[x].mean() for x in range(3)])
# print([std[x].mean() for x in range(3)])

AttributeError: ignored

# Neural Network

In [0]:
from torch import nn
import torch.nn.functional as F

class BoatNetwork(nn.Module):
  def __init__(self, num_classes=9):
    super(BoatNetwork, self).__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=2, padding=1).double()
    self.conv2 = nn.Conv2d(16, 32, kernel_size=2, padding=1).double()
    self.conv3 = nn.Conv2d(32, 64, kernel_size=2, padding=1).double()
    self.fc = nn.Linear(64*64*64, 64)
    self.fc2 = nn.Linear(64, num_classes)
  def forward(self, x):
    x = x.to(device)
    x = x.double()
    x = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
    x = F.max_pool2d(torch.tanh(self.conv2(x)), 2)
    x = F.max_pool2d(torch.tanh(self.conv3(x)), 2)
    x = x.reshape(x.size(0), -1).float()
    x = torch.tanh(self.fc(x))
    x = self.fc2(x)
    return F.log_softmax(x)

In [0]:
model = BoatNetwork()
model.to(device)
learning_rate = 1e-2
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
loss_fn.to(device)
n_epochs = 100
model.cuda()

BoatNetwork(
  (conv1): Conv2d(3, 16, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (fc): Linear(in_features=262144, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=9, bias=True)
)

In [0]:
from tensorboardX import SummaryWriter
writer = SummaryWriter()
writer.add_scalar('data/loss', 12, 1)

In [0]:
loop_count_exists = os.path.isfile(os.path.join(base_path, 'EPOCH_COUNT'))
if loop_count_exists: # Resume training from earlier save
  with open(os.path.join(base_path, 'EPOCH_COUNT')) as epoch_count:
    epoch_number = int(epoch_count.read())
  model.load_state_dict(torch.load(os.path.join(base_path, 'model_weights.pt')))
else:
  epoch_number = 0 # Start from epoch 0
for epoch in range(epoch_number, n_epochs):
  i=0
  for imgs, labels in train_image_loader:
    imgs.to(device)
    labels.to(device)
    outputs = model(imgs)
    outputs.to(device)
    loss = loss_fn(outputs, labels.cuda())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    writer.add_scalar('data/loss', loss, epoch)
  
  print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
  torch.save(model.state_dict(), os.path.join(base_path, 'model_weights.pt'))
  with open(os.path.join(base_path, 'EPOCH_COUNT'), 'w') as epoch_count:
    epoch_count.write(str(epoch))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 27, Loss: 0.001406
Epoch: 28, Loss: 0.001229
Epoch: 29, Loss: 0.001112
